In [2]:
import pandas as pd

In [3]:
prev_month = 1
next_month = prev_month+1


In [3]:
data_path = f'../data/2025-{prev_month:02d}/T_ONTIME_MARKETING.csv'
df = pd.read_csv(data_path)

In [4]:
# Variable objetivo

In [5]:
TARGET = 'DEP_DELAY_15'
df[TARGET] = (df['DEP_DELAY'] > 15)*1

In [6]:
df[TARGET].mean()

np.float64(0.17072751342625286)

In [7]:
# Pretratamiento EDA
numeric_cols = ['DEP_DELAY', 'ARR_DELAY', 'TAXI_OUT', 'TAXI_IN', 
                'AIR_TIME', 'DISTANCE', 'ACTUAL_ELAPSED_TIME']
for c in numeric_cols:
    q95 = df[c].quantile(.95)
    df[c] = df[c].apply(lambda x: q95 if x > q95 else x)

In [8]:
# Variables de EDA

In [9]:

df['ruta'] = df['ORIGIN'] + '-' + df['DEST']
# Creación de variables de temporalidad
date_col = 'FL_DATE'
df[date_col] = pd.to_datetime(df[date_col])
df['MONTH'] = df[date_col].dt.month
df['YEAR'] = df[date_col].dt.year
df['QUARTER'] = df[date_col].dt.quarter
df['DOW'] = df[date_col].dt.dayofweek # 0=Lunes
df['DOW_NAME'] = df[date_col].dt.day_name()
df['IS_WEEKEND'] = (df['DOW'] >= 5).astype(int)

# Crear flags basados en valores positivos/negativos
# Salida
df['FLAG_DELAYED_DEP'] = (df['DEP_DELAY'] > 15).astype(int)  # Retraso significativo >15 min
df['FLAG_EARLY_DEP'] = (df['DEP_DELAY'] < -5).astype(int)  # Salida temprana

# Llegada
df['FLAG_DELAYED_ARR'] = (df['ARR_DELAY'] > 15).astype(int)  # Llegada tardía >15 min
df['FLAG_EARLY_ARR'] = (df['ARR_DELAY'] < -5).astype(int)  # Llegada temprana
df['FLAG_SEVERE_DELAY'] = (df['ARR_DELAY'] > 60).astype(int)  # Retraso severo >1h

df['FLAG_LONG_TAXI_OUT'] = (df['TAXI_OUT'] > df['TAXI_OUT'].quantile(0.75)).astype(int)

# Oportunidad para comunicar eficiencia operacional (llegada antes de lo estimado)
df['FLAG_FAST_FLIGHT'] = ((df['ACTUAL_ELAPSED_TIME'] < df['CRS_ELAPSED_TIME'])).astype(int)

# Escenarios de negocio
# Cancelaciones y desvíos afectan confiabilidad de marca
df['FLAG_OPERATIONAL_ISSUE'] = ((df['CANCELLED'] == 1) | (df['DIVERTED'] == 1)).astype(int)

# Análisis de eficiencia (tiempo real vs programado)
df['ACTUAL_VS_CRS_TIME_DIFF'] = df['ACTUAL_ELAPSED_TIME'] - df['CRS_ELAPSED_TIME']

# Porcentage de tiempo de TAXI OUT, AIR TIEME, TAXI IN
for c in ['TAXI_OUT', 'AIR_TIME', 'TAXI_IN']:
    df[c+'_pct'] = df[c] / df['ACTUAL_ELAPSED_TIME']

C:\Users\PC\AppData\Local\Temp\ipykernel_22128\1272941147.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[date_col] = pd.to_datetime(df[date_col])


In [10]:
df.columns

Index(['FL_DATE', 'MKT_CARRIER_FL_NUM', 'OP_UNIQUE_CARRIER', 'TAIL_NUM',
       'OP_CARRIER_FL_NUM', 'ORIGIN', 'DEST', 'CRS_DEP_TIME', 'DEP_TIME',
       'DEP_DELAY', 'DEP_DELAY_NEW', 'DEP_DEL15', 'TAXI_OUT', 'WHEELS_OFF',
       'WHEELS_ON', 'TAXI_IN', 'CRS_ARR_TIME', 'ARR_TIME', 'ARR_DELAY',
       'ARR_DELAY_NEW', 'CANCELLED', 'CANCELLATION_CODE', 'DIVERTED',
       'CRS_ELAPSED_TIME', 'ACTUAL_ELAPSED_TIME', 'AIR_TIME', 'FLIGHTS',
       'DISTANCE', 'CARRIER_DELAY', 'WEATHER_DELAY', 'NAS_DELAY',
       'SECURITY_DELAY', 'LATE_AIRCRAFT_DELAY', 'TOTAL_ADD_GTIME',
       'LONGEST_ADD_GTIME', 'DEP_DELAY_15', 'ruta', 'MONTH', 'YEAR', 'QUARTER',
       'DOW', 'DOW_NAME', 'IS_WEEKEND', 'FLAG_DELAYED_DEP', 'FLAG_EARLY_DEP',
       'FLAG_DELAYED_ARR', 'FLAG_EARLY_ARR', 'FLAG_SEVERE_DELAY',
       'FLAG_LONG_TAXI_OUT', 'FLAG_FAST_FLIGHT', 'FLAG_OPERATIONAL_ISSUE',
       'ACTUAL_VS_CRS_TIME_DIFF', 'TAXI_OUT_pct', 'AIR_TIME_pct',
       'TAXI_IN_pct'],
      dtype='object')

In [11]:
numeric_cols = [
    'CRS_DEP_TIME', 'DEP_TIME', 'DEP_DELAY', 'TAXI_OUT',
    'TAXI_IN', 'CRS_ARR_TIME', 'ARR_TIME', 'ARR_DELAY',
    'CRS_ELAPSED_TIME',
    'ACTUAL_ELAPSED_TIME', 'AIR_TIME', 
    'FLIGHTS', 'DISTANCE',
       # 'CARRIER_DELAY', 'WEATHER_DELAY', 'NAS_DELAY', 'SECURITY_DELAY',
       # 'LATE_AIRCRAFT_DELAY', 'FIRST_DEP_TIME', 'TOTAL_ADD_GTIME',
       # 'LONGEST_ADD_GTIME', 'DIV_AIRPORT_LANDINGS', 'DIV_ACTUAL_ELAPSED_TIME', 'DIV_ARR_DELAY', 'DIV_DISTANCE',
       # 'DIV1_AIRPORT_ID',  'FLAG_DELAYED_DEP',
    'ACTUAL_VS_CRS_TIME_DIFF', 'TAXI_OUT_pct',
    'AIR_TIME_pct', 'TAXI_IN_pct']

flags_cols = [
    'CANCELLED', 'DIVERTED', 'FLAG_EARLY_DEP', 'FLAG_DELAYED_ARR', 'FLAG_EARLY_ARR',
       'FLAG_SEVERE_DELAY', 'FLAG_LONG_TAXI_OUT', 'FLAG_FAST_FLIGHT',
       'FLAG_OPERATIONAL_ISSUE', 
]

cat_cols = [
    'ruta', 'MONTH',
       'YEAR', 'QUARTER', 'DOW', 'DOW_NAME', 'IS_WEEKEND',
]

In [12]:
df.columns

Index(['FL_DATE', 'MKT_CARRIER_FL_NUM', 'OP_UNIQUE_CARRIER', 'TAIL_NUM',
       'OP_CARRIER_FL_NUM', 'ORIGIN', 'DEST', 'CRS_DEP_TIME', 'DEP_TIME',
       'DEP_DELAY', 'DEP_DELAY_NEW', 'DEP_DEL15', 'TAXI_OUT', 'WHEELS_OFF',
       'WHEELS_ON', 'TAXI_IN', 'CRS_ARR_TIME', 'ARR_TIME', 'ARR_DELAY',
       'ARR_DELAY_NEW', 'CANCELLED', 'CANCELLATION_CODE', 'DIVERTED',
       'CRS_ELAPSED_TIME', 'ACTUAL_ELAPSED_TIME', 'AIR_TIME', 'FLIGHTS',
       'DISTANCE', 'CARRIER_DELAY', 'WEATHER_DELAY', 'NAS_DELAY',
       'SECURITY_DELAY', 'LATE_AIRCRAFT_DELAY', 'TOTAL_ADD_GTIME',
       'LONGEST_ADD_GTIME', 'DEP_DELAY_15', 'ruta', 'MONTH', 'YEAR', 'QUARTER',
       'DOW', 'DOW_NAME', 'IS_WEEKEND', 'FLAG_DELAYED_DEP', 'FLAG_EARLY_DEP',
       'FLAG_DELAYED_ARR', 'FLAG_EARLY_ARR', 'FLAG_SEVERE_DELAY',
       'FLAG_LONG_TAXI_OUT', 'FLAG_FAST_FLIGHT', 'FLAG_OPERATIONAL_ISSUE',
       'ACTUAL_VS_CRS_TIME_DIFF', 'TAXI_OUT_pct', 'AIR_TIME_pct',
       'TAXI_IN_pct'],
      dtype='object')

In [13]:
f'{df.memory_usage(deep=True).sum() / (1024**3)} GB'

'0.42997891549021006 GB'

In [14]:
# Guardamos agrupados

In [15]:
# Variables flag
l_time_dimension = ['DOW', 'IS_WEEKEND'] # Día de la semana, Fin de semana vs Entre semana
l_dimension = ['OP_UNIQUE_CARRIER', 'ORIGIN', 'DEST'] # Aereolinea, Ruta, Aereopuerto Origen, Aereopuerto Destino
l_flag_frames = []

for dimension in l_dimension:
    for time_dimension in l_time_dimension:
        
        #file_path = f'temp/flag_{dimension}_{time_dimension}.csv'
        #l_flag_frames.append((file_path, dimension, time_dimension))
        #continue
        print(f'Dimensión: {dimension}, Temporalidad: {time_dimension}')
        
        # Agrupado por dimension y temporalidad
        x = df[flags_cols + [dimension, time_dimension]].groupby([dimension, time_dimension]).mean()
        x.columns = [c[0]+'_'+c[1] for c in x.columns]
        features = x.columns[:]
        x = x.reset_index()

        # Generación de combinación de columnas, pivot valores de agrupado de temporalidad por cada columna calculada
        x = x.pivot_table(index=dimension, columns=time_dimension, values=features)
        x.columns = [f'{dimension}_{c[0]}_{time_dimension}_{c[1]}' for c in x.columns]
        
        # df = df.merge(x, how='left', on=dimension)
        file_path = f'temp/flag_{dimension}_{time_dimension}.csv'
        l_flag_frames.append((file_path, dimension, time_dimension))
        
        x.to_csv(file_path, index=True)

Dimensión: OP_UNIQUE_CARRIER, Temporalidad: DOW
Dimensión: OP_UNIQUE_CARRIER, Temporalidad: IS_WEEKEND
Dimensión: ORIGIN, Temporalidad: DOW
Dimensión: ORIGIN, Temporalidad: IS_WEEKEND
Dimensión: DEST, Temporalidad: DOW
Dimensión: DEST, Temporalidad: IS_WEEKEND


In [16]:
# Variables continuas
l_time_dimension = ['IS_WEEKEND']
l_dimension = ['OP_UNIQUE_CARRIER', 'ORIGIN']
l_cont_frames = []

for dimension in l_dimension:
    for time_dimension in l_time_dimension:
        
        #file_path = f'temp/cont_{dimension}_{time_dimension}.csv'
        #l_cont_frames.append((file_path, dimension, time_dimension))
        #continue
        print(f'Dimensión: {dimension}, Temporalidad: {time_dimension}')
        
        # Agrupado por dimension y temporalidad
        x = df[numeric_cols + [dimension, time_dimension]].groupby([dimension, time_dimension]).agg(['mean', 'std', 'median'])#.drop('count', axis=1, level=1)
        x.columns = [c[0]+'_'+c[1] for c in x.columns]
        features = x.columns[:]
        x = x.reset_index()

        # Generación de combinación de columnas, pivot valores de agrupado de temporalidad por cada columna calculada
        x = x.pivot_table(index=dimension, columns=time_dimension, values=features)
        x.columns = [f'{dimension}_{c[0]}_{time_dimension}_{c[1]}' for c in x.columns]
        
        # df = df.merge(x, how='left', on=dimension)
        
        print(x.shape)
        file_path = f'temp/cont_{dimension}_{time_dimension}.csv'
        l_cont_frames.append((file_path, dimension, time_dimension))
        
        x.to_csv(file_path, index=True)

Dimensión: OP_UNIQUE_CARRIER, Temporalidad: IS_WEEKEND
(21, 102)
Dimensión: ORIGIN, Temporalidad: IS_WEEKEND
(352, 102)


In [17]:
# Delete data from previous month and load next month
del df


In [9]:
data_path = f'../data/2025-{next_month:02d}/T_ONTIME_MARKETING.csv'
df = pd.read_csv(data_path)

C:\Users\PC\AppData\Local\Temp\ipykernel_18676\1281906475.py:2: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_path)


In [10]:
df['ruta'] = df['ORIGIN'] + '-' + df['DEST']
# Creación de variables de temporalidad
date_col = 'FL_DATE'
df[date_col] = pd.to_datetime(df[date_col])
df['MONTH'] = df[date_col].dt.month
df['YEAR'] = df[date_col].dt.year
df['QUARTER'] = df[date_col].dt.quarter
df['DOW'] = df[date_col].dt.dayofweek # 0=Lunes
df['DOW_NAME'] = df[date_col].dt.day_name()
df['IS_WEEKEND'] = (df['DOW'] >= 5).astype(int)

# Crear flags basados en valores positivos/negativos
# Salida
df['FLAG_DELAYED_DEP'] = (df['DEP_DELAY'] > 15).astype(int)  # Retraso significativo >15 min
df['FLAG_EARLY_DEP'] = (df['DEP_DELAY'] < -5).astype(int)  # Salida temprana

# Llegada
df['FLAG_DELAYED_ARR'] = (df['ARR_DELAY'] > 15).astype(int)  # Llegada tardía >15 min
df['FLAG_EARLY_ARR'] = (df['ARR_DELAY'] < -5).astype(int)  # Llegada temprana
df['FLAG_SEVERE_DELAY'] = (df['ARR_DELAY'] > 60).astype(int)  # Retraso severo >1h

df['FLAG_LONG_TAXI_OUT'] = (df['TAXI_OUT'] > df['TAXI_OUT'].quantile(0.75)).astype(int)

# Oportunidad para comunicar eficiencia operacional (llegada antes de lo estimado)
df['FLAG_FAST_FLIGHT'] = ((df['ACTUAL_ELAPSED_TIME'] < df['CRS_ELAPSED_TIME'])).astype(int)

# Escenarios de negocio
# Cancelaciones y desvíos afectan confiabilidad de marca
df['FLAG_OPERATIONAL_ISSUE'] = ((df['CANCELLED'] == 1) | (df['DIVERTED'] == 1)).astype(int)

# Análisis de eficiencia (tiempo real vs programado)
df['ACTUAL_VS_CRS_TIME_DIFF'] = df['ACTUAL_ELAPSED_TIME'] - df['CRS_ELAPSED_TIME']

# Porcentage de tiempo de TAXI OUT, AIR TIEME, TAXI IN
for c in ['TAXI_OUT', 'AIR_TIME', 'TAXI_IN']:
    df[c+'_pct'] = df[c] / df['ACTUAL_ELAPSED_TIME']

C:\Users\PC\AppData\Local\Temp\ipykernel_18676\895551814.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[date_col] = pd.to_datetime(df[date_col])


In [ ]:
# Variables que sí se usarán del mes a predecir
TARGET = 'DEP_DELAY_15'
df[TARGET] = (df['DEP_DELAY'] > 15)*1

ID_COLS = ['FL_DATE', 'OP_UNIQUE_CARRIER','TAIL_NUM', 'ORIGIN', 'DEST',]
dummies = ['OP_UNIQUE_CARRIER','TAIL_NUM']
schedule_cols = ['CRS_DEP_TIME', 'CRS_ARR_TIME', 'CRS_ELAPSED_TIME','IS_WEEKEND',]

df = df[[TARGET] + ID_COLS + schedule_cols]       

In [ ]:
# Crear dataframe para modelo

for p, d, t in l_flag_frames+l_cont_frames:
    temp_frame = pd.read_csv(p)
    print(p, temp_frame.shape)
    df = df.merge(temp_frame, on=d, how='left')
    print(df.shape)

temp/flag_OP_UNIQUE_CARRIER_DOW.csv (21, 64)
(559577, 73)
temp/flag_OP_UNIQUE_CARRIER_IS_WEEKEND.csv (21, 19)
(559577, 91)
temp/flag_ORIGIN_DOW.csv (352, 64)
(559577, 154)
temp/flag_ORIGIN_IS_WEEKEND.csv (352, 19)
(559577, 172)
temp/flag_DEST_DOW.csv (352, 64)
(559577, 235)
temp/flag_DEST_IS_WEEKEND.csv (352, 19)
(559577, 253)
temp/cont_OP_UNIQUE_CARRIER_IS_WEEKEND.csv (21, 103)
(559577, 355)
temp/cont_ORIGIN_IS_WEEKEND.csv (352, 103)
(559577, 457)


In [ ]:
pd.get_dummies(df)

In [22]:
df.to_csv(f'X_2025-{next_month:02d}.csv', index=False)

In [23]:
1+1

2